<a href="https://colab.research.google.com/github/paurbano/aplicaciones-financieras/blob/main/Taller_final_Modulo_2_Aplicaciones_financieras_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librerias

In [ ]:
!pip install -U plotly

In [18]:
import pandas as pd
import numpy as np
import tensorflow.keras as K

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime

#Datos
Variable objetivo: costo del seguro

##Cargar Dataset

In [21]:
data = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [6]:
data.describe()
#data.info()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


#Preprocesamiento DataSet

## Transformar datos categoricos


In [ ]:
# seleccionamos solo los datos categoricos
cat = data.select_dtypes(include=[object])
# cat.head(3)
cat.shape

### Codificación de variables categoricas

In [22]:
le_sex = LabelEncoder()
le_smoker = LabelEncoder()
le_region = LabelEncoder()
data['sex_encoded'] = le_sex.fit_transform(data.sex)
data['smoker_encoded'] = le_smoker.fit_transform(data.smoker)
data['region_encoded'] = le_region.fit_transform(data.region)
data.head(5)

,age,sex,bmi,children,smoker,region,charges,sex_encoded,smoker_encoded,region_encoded
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,3
1,18,male,33.770,1,no,southeast,1725.55230,1,0,2
2,28,male,33.000,3,no,southeast,4449.46200,1,0,2
3,33,male,22.705,0,no,northwest,21984.47061,1,0,1
4,32,male,28.880,0,no,northwest,3866.85520,1,0,1


In [25]:
# sex_ohe = OneHotEncoder()
# smoker_ohe = OneHotEncoder()
region_ohe = OneHotEncoder()
region_fit = region_ohe.fit_transform(data.region_encoded.values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(region_fit, columns=["region_"+str(int(i)) for i in range(region_fit.shape[1])])
data = pd.concat([data,dfOneHot], axis=1)


In [26]:
data

,age,sex,bmi,children,smoker,region,charges,sex_encoded,smoker_encoded,region_encoded,region_0,region_1,region_2,region_3
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,3,0.0,0.0,0.0,1.0
1,18,male,33.770,1,no,southeast,1725.55230,1,0,2,0.0,0.0,1.0,0.0
2,28,male,33.000,3,no,southeast,4449.46200,1,0,2,0.0,0.0,1.0,0.0
3,33,male,22.705,0,no,northwest,21984.47061,1,0,1,0.0,1.0,0.0,0.0
4,32,male,28.880,0,no,northwest,3866.85520,1,0,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,1,0,1,0.0,1.0,0.0,0.0
1334,18,female,31.920,0,no,northeast,2205.98080,0,0,0,1.0,0.0,0.0,0.0
1335,18,female,36.850,0,no,southeast,1629.83350,0,0,2,0.0,0.0,1.0,0.0
1336,21,female,25.800,0,no,southwest,2007.94500,0,0,3,0.0,0.0,0.0,1.0


In [27]:
preprocessData = data.drop(columns=['sex','smoker','region','region_encoded'])
preprocessData

,age,bmi,children,charges,sex_encoded,smoker_encoded,region_0,region_1,region_2,region_3
0,19,27.900,0,16884.92400,0,1,0.0,0.0,0.0,1.0
1,18,33.770,1,1725.55230,1,0,0.0,0.0,1.0,0.0
2,28,33.000,3,4449.46200,1,0,0.0,0.0,1.0,0.0
3,33,22.705,0,21984.47061,1,0,0.0,1.0,0.0,0.0
4,32,28.880,0,3866.85520,1,0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,0.0,1.0,0.0,0.0
1334,18,31.920,0,2205.98080,0,0,1.0,0.0,0.0,0.0
1335,18,36.850,0,1629.83350,0,0,0.0,0.0,1.0,0.0
1336,21,25.800,0,2007.94500,0,0,0.0,0.0,0.0,1.0
